In [1]:
import QBOOST as qb
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import math
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

In [3]:
data = pd.read_csv('train.csv')
y = data['target']
x = data.drop(['target', 'ID_code'], axis=1)

In [4]:
# Using PCA to reduce the dimensionality of the data
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
pca.fit(x)
x_pca = pca.transform(x)

In [5]:
# Reducing the size of the data
x, _, y, _ = train_test_split(x_pca, y, test_size=0.5, random_state=42)

In [6]:
# Splitting the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [8]:
# Finding the best params for the knn classifier

params = {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]}
knn = KNeighborsClassifier()
clf = GridSearchCV(knn, params, cv=5)
clf.fit(x_train, y_train)
print(clf.best_params_)
print(clf.best_score_)

best_number_of_neighbors = clf.best_params_['n_neighbors']

{'n_neighbors': 17}
0.9001250000000001


In [9]:
# Finding the best params for the decision tree classifier

params = {'max_depth': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]}
dt = DecisionTreeClassifier()
clf = GridSearchCV(dt, params, cv=5)
clf.fit(x_train, y_train)
print(clf.best_params_)
print(clf.best_score_)

best_depth = clf.best_params_['max_depth']

{'max_depth': 1}
0.9001374999999999


In [10]:
# Training 30 weak learners knn and decision trees
weak_learners = 30
L = []

for i in range(weak_learners):
    clf = KNeighborsClassifier(n_neighbors=best_number_of_neighbors)
    clf.fit(x_train, y_train)
    L.append(clf)

for j in range(weak_learners):
    clf = DecisionTreeClassifier(max_depth=best_depth)
    clf.fit(x_train, y_train)
    L.append(clf)

In [11]:
w_opt = qb.RGS(L,x_train,y_train,1)
y_pred = qb.PredictRGS(L,w_opt,x_test)

KeyboardInterrupt: 

In [ ]:
w_opt

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, cmap="Blues", fmt="d")

In [ ]:
# Plotting the PR curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import auc
average_precision = average_precision_score(y_test, y_pred)
print('Average precision-recall score: {0:0.2f}'.format(average_precision))


from sklearn.metrics import precision_recall_curve
from funcsigs import signature

precision, recall, _ = precision_recall_curve(y_test, y_pred)
aucpr_score = auc(recall, precision)

step_kwargs = ({'step': 'post'}
                if 'step' in signature(plt.fill_between).parameters
                else {})
plt.step(recall, precision, color='b', alpha=0.2,
            where='post')   
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('PR curve for RGS (n=60, maxd=5): AP={0:0.2f}'.format(average_precision) + ' ' + 'AUCPR={0:0.2f}'.format(aucpr_score))